# Notebook Final del PFM
## Título: Comparativa de algoritmos de clustering sobre datos de futbolistas de las grandes ligas europeas.
## Tutor: Miguel Camacho

Los datos se han obtenido de fbref.com. Están compuestos de 128 jugadores ofensivos que han participado en las últimas cinco temporadas de forma ininterrumpida en las cinco grandes ligas europeas de fútbol (Premier League, LaLiga, Serie A, Bundesliga y Ligue 1). 



# Importar Librerías

In [ ]:
!pip install pandas-profiling==2.7.1
!pip install pandas==1.2.4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import sys, getopt
import csv
import pandas_profiling
import numpy as np
from sklearn import preprocessing
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Definir constantes y funciones auxiliares

In [ ]:
class Summary:
  START_HEADER = 9
  N_COLUMNS = 38
  END_HEADER = START_HEADER + N_COLUMNS
  FIELDS_SUMMARY = ['Date','Day','Comp','Round','Venue','Result',
                    'Squad','Opponent','Start','Pos','Min','Gls','Ast','PK',
                    'PKatt','Sh','SoT','CrdY','CrdR','Touches','Press','Tkl',
                    'Int','Blocks','xG','npxG','xA','SCA','GCA','Cmp','Att',
                    'CmpPerc','Prog','Carries','Prog','Succ','Att',
                    'Match Report','Player']
  FIELDS_MERGE_SUMMARY = ['Gls', 'PK', 'PKatt', 'Sh', 'SoT', 'xG', 'npxG']

class Passing:
  START_HEADER = 8
  N_COLUMNS = 33
  END_HEADER = START_HEADER + N_COLUMNS
  FIELDS_PASSING = ['Date','Day','Comp','Round','Venue','Result',
                    'Squad','Opponent','Start','Pos','Min','TotCmp','TotAtt',
                    'TotCmpPerc','TotDist','PrgDist','ShCmp','ShAtt',
                    'ShCmpPerc','MedCmp','MedAtt','MedCmpPerc','LongCmp',
                    'LongAtt','LongCmpPerc','Ast','xA','KP','1/3','PPA','CrsPA',
                    'Prog','Match Report','Player']

class Pass_Types:
  START_HEADER = 9
  N_COLUMNS = 37
  END_HEADER = START_HEADER + N_COLUMNS
  FIELDS_PASS_TYPES = ['Date','Day','Comp','Round','Venue','Result',
                       'Squad','Opponent','Start','Pos','Min','Att','Live',
                       'Dead','FK','TB','Press','Sw','Crs','CK','InCK','OutCK',
                       'StrCK','GroundPass','LowPass','HighPass','LeftPass',
                       'RightPass','HeadPass','TI','Other','CmpPass','OffSide',
                       'Out','IntPass','BlockPass','Match Report','Player']

class GCA:
  START_HEADER = 5
  N_COLUMNS = 26
  END_HEADER = START_HEADER + N_COLUMNS
  FIELDS_GCA = ['Date','Day','Comp','Round','Venue','Result','Squad',
                'Opponent','Start','Pos','Min','SCA','PassLiveSCA',
                'PassDeadSCA','DribSCA','ShSCA','FldSCA','DefSCA','GCA',
                'PassLiveGCA','PassDeadGCA','DribGCA','ShGCA','FldGCA',
                'DefGCA','Match Report','Player']

class Defensive:
  START_HEADER = 7
  N_COLUMNS = 35
  END_HEADER = START_HEADER + N_COLUMNS
  FIELDS_DEFENSIVE = ['Date','Day','Comp','Round','Venue','Result',
                      'Squad','Opponent','Start','Pos','Min','Tkl','TklW',
                      'Def3rdTkl','Mid3rdTkl','Att3rdTkl','TklDrib',
                      'TklDribAtt','TklDribPerc','TklDribPast','PressAtt',
                      'PressSucc','PressPerc','Def3rdPress','Mid3rdPress',
                      'Att3rdPress','Blocks','ShBlock','ShSvBlock','PassBlock',
                      'Int','Tkl+Int','Clr','Err','Match Report','Player']

class Possession:
  START_HEADER = 7
  N_COLUMNS = 36
  END_HEADER = START_HEADER + N_COLUMNS
  FIELDS_POSSESSION = ['Date','Day','Comp','Round','Venue','Result',
                       'Squad','Opponent','Start','Pos','Min','Touches',
                       'DefPenTouches','Def3rdTouches','Mid3rdTouches',
                       'Att3rdTouches','AttPenTouches','LiveTouches',
                       'SuccDrib','AttDrib','SuccDribPerc','NuPlDrib',
                       'Nutmegs','Carries','TotDistCarries','PrgDistCarries',
                       'ProgCarries','1/3Carries','CPA','MisCarries',
                       'DisCarries','TargPass','RecPass','RecPassPerc',
                       'ProgPassRec','Match Report','Player']

class Miscellaneous:
  START_HEADER = 5
  N_COLUMNS = 28
  END_HEADER = START_HEADER + N_COLUMNS
  FIELDS_MISC = ['Date','Day','Comp','Round','Venue','Result','Squad',
                 'Opponent','Start','Pos','Min','CrdY','CrdR','2CrdY','FlsComm',
                 'FlsDrawn','Offsides','Crs','Int','TklW','PKwon','PKcon','OG',
                 'Recov','AerialDuelsWon','AerialDuelsLost',
                 'AerialDuelsWinPerc','Match Report', 'Player']
  FIELDS_MERGE_MISC = ['CrdY', 'CrdR', '2CrdY', 'FlsComm', 'FlsDrawn', 
                       'Offsides', 'PKwon', 'PKcon', 'OG', 'Recov', 
                       'AerialDuelsWon', 'AerialDuelsLost', 
                       'AerialDuelsWinPerc', 'Match Report']


class Global:
  START_DATA = 2
  KEY = ['Player', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 
       'Opponent', 'Start', 'Pos', 'Min']
  TYPES = ['summary', 'passing', 'passing_types', 'gca', 'defense', 
           'possession', 'misc']
  DICT_TYPES = {'summary': Summary, 'passing': Passing, 
                'passing_types': Pass_Types, 'gca': GCA, 
                'defense': Defensive, 'possession': Possession, 
                'misc': Miscellaneous}
  SEASONS = ['2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018']
  DICT_PLAYERS = {
      'Karim Benzema': 'https://fbref.com/en/players/70d74ece/matchlogs/',
      'Lionel Messi': 'https://fbref.com/en/players/d70ce98e/matchlogs/',
      'Kylian Mbappe': 'https://fbref.com/en/players/42fd9c7f/matchlogs/',
      'Memphis Depay': 'https://fbref.com/en/players/8f696594/matchlogs/',
      'Duvan Zapata': 'https://fbref.com/en/players/d3de9af0/matchlogs/',
      'Luis Muriel': 'https://fbref.com/en/players/eb2fe5b6/matchlogs/',
      'Robert Lewandowski': 'https://fbref.com/en/players/8d78e732/matchlogs/',
      'Gerard Moreno': 'https://fbref.com/en/players/81f0781e/matchlogs/',
      'Cristiano Ronaldo': 'https://fbref.com/en/players/dea698d9/matchlogs/',
      'Harry Kane': 'https://fbref.com/en/players/21a66f6a/matchlogs/',
      'Paulo Dybala': 'https://fbref.com/en/players/e0921a4f/matchlogs/',
      'Antoine Griezmann': 'https://fbref.com/en/players/df69b544/matchlogs/',
      'Roberto Firmino': 'https://fbref.com/en/players/4c370d81/matchlogs/',
      'Dimitri Payet': 'https://fbref.com/en/players/58ae47b2/matchlogs/',
      'Iago Aspas': 'https://fbref.com/en/players/7dcf86f6/matchlogs/',
      'Marco Reus': 'https://fbref.com/en/players/36a3ff67/matchlogs/',
      'Kingsley Coman': 'https://fbref.com/en/players/042e8a49/matchlogs/',
      'Angel Di Maria': 'https://fbref.com/en/players/19cda00b/matchlogs/',
      'Leroy Sane': 'https://fbref.com/en/players/2b114be3/matchlogs/',
      'Neymar': 'https://fbref.com/en/players/69384e5d/matchlogs/',
      'Riyad Mahrez': 'https://fbref.com/en/players/892d5bb1/matchlogs/',
      'Nabil Fekir': 'https://fbref.com/en/players/bece776f/matchlogs/',
      'Serge Gnabry': 'https://fbref.com/en/players/88e357ef/matchlogs/',
      'Erik Lamela': 'https://fbref.com/en/players/abe66106/matchlogs/',
      'Nicolas Pepe': 'https://fbref.com/en/players/57e3f0c7/matchlogs/',
      'Christopher Nkunku': 'https://fbref.com/en/players/7c56da38/matchlogs/',
      'Phil Foden': 'https://fbref.com/en/players/ed1e53f3/matchlogs/',
      'Roberto Firmino': 'https://fbref.com/en/players/4c370d81/matchlogs/',
      'Dimitri Payet': 'https://fbref.com/en/players/58ae47b2/matchlogs/',
      'Angel Correa': 'https://fbref.com/en/players/01eb744d/matchlogs/',
      'Federico Chiesa': 'https://fbref.com/en/players/b0f7e36c/matchlogs/',
      'Sadio Mane': 'https://fbref.com/en/players/c691bfe2/matchlogs/',
      'Wissam Ben Yedder': 'https://fbref.com/en/players/942b4f90/matchlogs/',
      'Breel Embolo': 'https://fbref.com/en/players/0b4f388a/matchlogs/',
      'Alexandre Lacazette': 'https://fbref.com/en/players/9dbb75ca/matchlogs/',
      'Alvaro Morata': 'https://fbref.com/en/players/129af0db/matchlogs/',
      'Gabriel Jesus': 'https://fbref.com/en/players/b66315ae/matchlogs/',
      'Ciro Immobile': 'https://fbref.com/en/players/4431aed2/matchlogs/',
      'Romelu Lukaku': 'https://fbref.com/en/players/5eae500a/matchlogs/',
      'Edin Dzeko': 'https://fbref.com/en/players/3bb7f478/matchlogs/',
      'Andre Silva': 'https://fbref.com/en/players/3effaa34/matchlogs/',
      'Timo Werner':'https://fbref.com/en/players/49fe9070/matchlogs/',
      'Patrik Schick': 'https://fbref.com/en/players/5d4f7d61/matchlogs/',
      'Andy Delort': 'https://fbref.com/en/players/d33c706e/matchlogs/',
      'Luis Suarez': 'https://fbref.com/en/players/a6154613/matchlogs/',
      'Carlos Bacca': 'https://fbref.com/en/players/09a9e921/matchlogs/',
      'Rodrigo': 'https://fbref.com/en/players/1fb1c435/matchlogs/',
      'Gareth Bale': 'https://fbref.com/en/players/a58bb1e1/matchlogs/',
      'Antonio Sanabria': 'https://fbref.com/en/players/0a447501/matchlogs/',
      'Kike': 'https://fbref.com/en/players/e897d8ba/matchlogs/',
      'Inaki Williams': 'https://fbref.com/en/players/6a99e0b1/matchlogs/',
      'Jamie Vardy': 'https://fbref.com/en/players/45963054/matchlogs/',
      'Paco Alcacer': 'https://fbref.com/en/players/a7a9d95a/matchlogs/',
      'Edinson Cavani': 'https://fbref.com/en/players/527f063d/matchlogs/',
      'Willian Jose': 'https://fbref.com/en/players/d87e2cae/matchlogs/',
      'Joao Pedro': 'https://fbref.com/en/players/81255c03/matchlogs/',
      'Kevin Lasagna': 'https://fbref.com/en/players/09538fdb/matchlogs/',
      'Fabio Quagliarella': 'https://fbref.com/en/players/ee4f2f3b/matchlogs/',
      'Lucas Perez': 'https://fbref.com/en/players/a300ac7e/matchlogs/',
      'Manolo Gabbiadini': 'https://fbref.com/en/players/8f866fe8/matchlogs/',
      'Roger Marti': 'https://fbref.com/en/players/0ae4e09a/matchlogs/',
      'Danny Ings': 'https://fbref.com/en/players/07802f7f/matchlogs/',
      'Ruben Sobrino': 'https://fbref.com/en/players/19b776e9/matchlogs/',
      'Raul Garcia': 'https://fbref.com/en/players/b418dbd4/matchlogs/',
      'Richarlison': 'https://fbref.com/en/players/fa031b34/matchlogs/',
      'Maxi Gomez': 'https://fbref.com/en/players/4c2e9442/matchlogs/',
      'Jorge Molina': 'https://fbref.com/en/players/43f71e77/matchlogs/',
      'Joselu': 'https://fbref.com/en/players/6265208f/matchlogs/',
      'Mattia Destro': 'https://fbref.com/en/players/d7d32194/matchlogs/',
      'Youssef En-Nesyri': 'https://fbref.com/en/players/04e17fd5/matchlogs/',
      'Olivier Giroud': 'https://fbref.com/en/players/16ceb862/matchlogs/',
      'Santi Mina': 'https://fbref.com/en/players/0b90bb97/matchlogs/',
      'Chris Wood': 'https://fbref.com/en/players/4e9a0555/matchlogs/',
      'Enes Unal': 'https://fbref.com/en/players/f8eca1b6/matchlogs/',
      'Leonardo Pavoletti': 'https://fbref.com/en/players/d37b0350/matchlogs/',
      'Giovanni Simeone': 'https://fbref.com/en/players/343c0d52/matchlogs/',
      'Christian Benteke': 'https://fbref.com/en/players/ab070c55/matchlogs/',
      'Dominic Calvert Lewin': 'https://fbref.com/en/players/59e6e5bf/matchlogs/',
      'Callum Wilson': 'https://fbref.com/en/players/c596fcb0/matchlogs/',
      'Keita Balde': 'https://fbref.com/en/players/509a4ccb/matchlogs/',
      'Edinson Cavani': 'https://fbref.com/en/players/527f063d/matchlogs/',
      'Mauro Icardi': 'https://fbref.com/en/players/43b78598/matchlogs/',
      'Arkadiusz Milik': 'https://fbref.com/en/players/85613cf0/matchlogs/',
      'Sehrou Guirassy': 'https://fbref.com/en/players/923f4dda/matchlogs/',
      'Angel Rodriguez': 'https://fbref.com/en/players/8cfc2f69/matchlogs/',
      'Munir-El-Haddadi': 'https://fbref.com/en/players/8696bc90/matchlogs/',
      'Mikel Oyarzabal': 'https://fbref.com/en/players/8c3c640c/matchlogs/',
      'Alex Berenguer': 'https://fbref.com/en/players/dc1c2fce/matchlogs/',
      'Henrikh Mkhitaryan': 'https://fbref.com/en/players/dd0daf32/matchlogs/',
      'Ivan Perisic': 'https://fbref.com/en/players/6fe90922/matchlogs/',
      'Pedro': 'https://fbref.com/en/players/3ca7254a/matchlogs/',
      'Raheem Sterling': 'https://fbref.com/en/players/b400bde0/matchlogs/',
      'Federico Bernardeschi': 'https://fbref.com/en/players/ee93c1a9/matchlogs/',
      'Antonio Candreva': 'https://fbref.com/en/players/356c9002/matchlogs/',
      'Hakan Calhanoglu': 'https://fbref.com/en/players/cd0fa27b/matchlogs/',
      'Jadon Sancho': 'https://fbref.com/en/players/dbf053da/matchlogs/',
      'Lorenzo Insigne': 'https://fbref.com/en/players/2f557579/matchlogs/',
      'Adnan Januzaj': 'https://fbref.com/en/players/4737cebe/matchlogs/',
      'Nathan Redmond': 'https://fbref.com/en/players/ab651565/matchlogs/',
      'Vincenzo Grifo': 'https://fbref.com/en/players/54e4866f/matchlogs/',
      'Iker Muniain': 'https://fbref.com/en/players/c05dfb74/matchlogs/',
      'Jordan Ayew': 'https://fbref.com/en/players/da052c14/matchlogs/',
      'Lucas Ocampos': 'https://fbref.com/en/players/a08b974a/matchlogs/',
      'Ludovic Blas': 'https://fbref.com/en/players/6191093d/matchlogs/',
      'Wilfried Zaha': 'https://fbref.com/en/players/b2bc3b1f/matchlogs/',
      'Lucas Moura': 'https://fbref.com/en/players/2b622f01/matchlogs/',
      'Papu Gomez': 'https://fbref.com/en/players/6e4df551/matchlogs/',
      'Suso': 'https://fbref.com/en/players/4e219ad2/matchlogs/',
      'Dele Alli': 'https://fbref.com/en/players/cea4ee8f/matchlogs/',
      'Xherdan Shaqiri': 'https://fbref.com/en/players/6421ec64/matchlogs/',
      'Son Heung-min': 'https://fbref.com/en/players/92e7e919/matchlogs/',
      'Mohamed Salah': 'https://fbref.com/en/players/e342ad68/matchlogs/',
      'Marcus Rashford': 'https://fbref.com/en/players/a1d5bd30/matchlogs/',
      'Ferran Torres': 'https://fbref.com/en/players/9e1035f8/matchlogs/',
      'Juanmi': 'https://fbref.com/en/players/84399660/matchlogs/',
      'Jose Luis Morales': 'https://fbref.com/en/players/4a478107/matchlogs/',
      'Kevin Volland': 'https://fbref.com/en/players/64f69877/matchlogs/',
      'Portu': 'https://fbref.com/en/players/1bda5842/matchlogs/',
      'Felipe Caicedo': 'https://fbref.com/en/players/93b891d1/matchlogs/',
      'Kelechi Iheanacho': 'https://fbref.com/en/players/c92e1a31/matchlogs/',
      'Simone Zaza': 'https://fbref.com/en/players/9592289a/matchlogs/',
      'Pierre-Emerick Aubameyang': 'https://fbref.com/en/players/d5dd5f1f/matchlogs/',
      'Borja Mayoral': 'https://fbref.com/en/players/64e8ed6d/matchlogs/',
      'Joaquin Correa': 'https://fbref.com/en/players/45b9b619/matchlogs/',
      'Andrej Kramaric': 'https://fbref.com/en/players/603cb947/matchlogs/',
      'Sandro Ramirez': 'https://fbref.com/en/players/833fb62e/matchlogs/',
      'Martin Braithwaite': 'https://fbref.com/en/players/fd771f95/matchlogs/',
      'Mariano': 'https://fbref.com/en/players/5c4dc0ff/matchlogs/',
      'Stevan Jovetic': 'https://fbref.com/en/players/f36c432f/matchlogs/',
      'Eden Hazard ': 'https://fbref.com/en/players/a39bb753/matchlogs/',
      'Anthony Martial': 'https://fbref.com/en/players/8b788c01/matchlogs/'
  }

In [ ]:
## Scrape one type
def scrape(url, page_type):
  res = requests.get(url).text
  soup = BeautifulSoup(res, 'html.parser')
  table = soup.find('table', class_='min_width sortable stats_table min_width shade_zero')
  n_games = len(table.tbody.find_all('tr'))
  columns = []
  dates = []
  row = []
  for i, header in enumerate(table.find_all('th')):
    if i in range(page_type.START_HEADER, page_type.END_HEADER):
      columns.append(header.text)
    if i in range(page_type.END_HEADER, page_type.END_HEADER + n_games):
      dates.append(header.text)
  df = pd.DataFrame(columns=columns)
  for j, data in enumerate(table.find_all('tr')):
    if j in range(Global.START_DATA, Global.START_DATA + n_games):
      raw_data = data.find_all('td')
      for a in raw_data:
        row.append(a.text)
      row.insert(0, dates[j-Global.START_DATA])
      if len(row) == page_type.N_COLUMNS:
        new_row = pd.Series(row, index = df.columns)
        df = df.append(new_row, ignore_index=True)
      row.clear()
  df['Player'] = url.split('/')[-1].replace('-Match-Logs', '').replace('-',' ')
  return df

def get_result(result):
  if len(result.split(' ')) == 2:
    return result.split(' ')
  else:
    return [result[0], result[1:].replace(' ','')]

## Scrape all types
def scrape_all(player, season, global_url, types):
  urls = []
  keys_list = list(Global.DICT_TYPES)
  for typ in types:
    urls.append(global_url + season + '/' + typ + '/' + player.replace(' ','-') + '-Match-Logs')
  for i, url in enumerate(urls):
    if i == 0:
      df_summary = scrape(url, Global.DICT_TYPES[url.split('/')[8]])
      df_summary.rename(columns=dict(zip(df_summary.columns, 
                                         Summary.FIELDS_SUMMARY)), inplace=True)
    if i == 1:
      df_passing = scrape(url, Global.DICT_TYPES[url.split('/')[8]])
      df_passing.rename(columns=dict(zip(df_passing.columns, 
                                         Passing.FIELDS_PASSING)), inplace=True)
    if i == 2:
      df_pass_types = scrape(url, Global.DICT_TYPES[url.split('/')[8]])
      df_pass_types.rename(columns=dict(zip(df_pass_types.columns, 
                                      Pass_Types.FIELDS_PASS_TYPES)), 
                           inplace=True)
    if i == 3:
      df_gca = scrape(url, Global.DICT_TYPES[url.split('/')[8]])
      df_gca.rename(columns=dict(zip(df_gca.columns, 
                                     GCA.FIELDS_GCA)), inplace=True)
    if i == 4:
      df_defensive = scrape(url, Global.DICT_TYPES[url.split('/')[8]])
      df_defensive.rename(columns=dict(zip(
          df_defensive.columns, Defensive.FIELDS_DEFENSIVE)), inplace=True)
    if i == 5:
      df_possession = scrape(url, Global.DICT_TYPES[url.split('/')[8]])
      df_possession.rename(columns=dict(
          zip(df_possession.columns, Possession.FIELDS_POSSESSION)), 
          inplace=True)
    if i == 6:
      df_misc = scrape(url, Global.DICT_TYPES[url.split('/')[8]])
      df_misc.rename(columns=dict(zip(
          df_misc.columns, Miscellaneous.FIELDS_MISC)), inplace=True)

  df = df_summary[Global.KEY + Summary.FIELDS_MERGE_SUMMARY].merge(
                     df_passing, left_on=Global.KEY, right_on=Global.KEY)
  df = df.merge(df_pass_types, left_on=Global.KEY, right_on=Global.KEY)
  df = df.merge(df_gca, left_on=Global.KEY, right_on=Global.KEY)
  df = df.merge(df_defensive, left_on=Global.KEY, right_on=Global.KEY)
  df = df.merge(df_possession, left_on=Global.KEY, right_on=Global.KEY)
  df = df.merge(df_misc[Global.KEY + Miscellaneous.FIELDS_MERGE_MISC], 
                left_on=Global.KEY, right_on=Global.KEY)
  df = df.fillna(0)
  df['Start'] = df['Start'].apply(lambda x: 1 if x in ('Y', 'Y*') else 0)
  df['Opponent'] = df['Opponent'].apply(lambda x: x[2:] if x[0].islower() else x)
  df['Squad'] = df['Squad'].apply(lambda x: x[2:] if x[0].islower() else x)
  df['Result'] = df['Result'].apply(lambda x: get_result(x))
  df['Score'] = df['Result'].apply(lambda x: x[1])
  df['Result'] = df['Result'].apply(lambda x: x[0])
  df['Score'] = df['Score'].apply(lambda x: x.split('–'))
  df['Squad_Goals'] = df['Score'].apply(lambda x: x[0])
  df['Opponent_Goals'] = df['Score'].apply(lambda x: x[1])
  df = df.drop(columns=['Score', 'Match Report_x', 'Match Report_y'])
  return df

In [ ]:
# RESULTADO
def result(df_points, i):
  return 0.3 if df_points.loc[i,'Result'] == 1 else -0.3 if df_points.loc[i,'Result'] == 3 else 0

# PASES TOTALES
def tot_pass(df_points, i):
  if df_points.loc[i,'TotAtt'] <= 20:
    return 0
  elif (df_points.loc[i,'TotAtt'] > 20) & (df_points.loc[i,'TotAtt'] <= 40):
    if df_points.loc[i,'TotCmpPerc'] <= 50:
      return -0.1
    elif (df_points.loc[i,'TotCmpPerc'] > 50) & (df_points.loc[i,'TotCmpPerc'] <= 60):
      return -0.05
    elif (df_points.loc[i,'TotCmpPerc'] > 60) & (df_points.loc[i,'TotCmpPerc'] <= 70):
      return 0
    elif (df_points.loc[i,'TotCmpPerc'] > 70) & (df_points.loc[i,'TotCmpPerc'] <= 80):
      return 0.025
    elif (df_points.loc[i,'TotCmpPerc'] > 80) & (df_points.loc[i,'TotCmpPerc'] <= 90):
      return 0.05
    else:
      return 0.1
  elif (df_points.loc[i,'TotAtt'] > 40) & (df_points.loc[i,'TotAtt'] <= 60):
    if df_points.loc[i,'TotCmpPerc'] <= 50:
      return -0.2
    elif (df_points.loc[i,'TotCmpPerc'] > 50) & (df_points.loc[i,'TotCmpPerc'] <= 60):
      return -0.1
    elif (df_points.loc[i,'TotCmpPerc'] > 60) & (df_points.loc[i,'TotCmpPerc'] <= 70):
      return 0
    elif (df_points.loc[i,'TotCmpPerc'] > 70) & (df_points.loc[i,'TotCmpPerc'] <= 80):
      return 0.05
    elif (df_points.loc[i,'TotCmpPerc'] > 80) & (df_points.loc[i,'TotCmpPerc'] <= 90):
      return 0.1
    else:
      return 0.2
  else:
    if df_points.loc[i,'TotCmpPerc'] <= 50:
      return -0.3
    elif (df_points.loc[i,'TotCmpPerc'] > 50) & (df_points.loc[i,'TotCmpPerc'] <= 60):
      return -0.15
    elif (df_points.loc[i,'TotCmpPerc'] > 60) & (df_points.loc[i,'TotCmpPerc'] <= 70):
      return 0
    elif (df_points.loc[i,'TotCmpPerc'] > 70) & (df_points.loc[i,'TotCmpPerc'] <= 80):
      return 0.075
    elif (df_points.loc[i,'TotCmpPerc'] > 80) & (df_points.loc[i,'TotCmpPerc'] <= 90):
      return 0.15
    else:
      return 0.3

# PASES EN CORTO
def short_pass(df_points, i):
  if df_points.loc[i,'ShortAtt'] <= 10:
    return 0
  elif (df_points.loc[i,'ShortAtt'] > 10) & (df_points.loc[i,'ShortAtt'] <= 25):
    if df_points.loc[i,'ShortCmpPerc'] <= 50:
      return -0.05
    elif (df_points.loc[i,'ShortCmpPerc'] > 50) & (df_points.loc[i,'ShortCmpPerc'] <= 70):
      return -0.025
    elif (df_points.loc[i,'ShortCmpPerc'] > 70) & (df_points.loc[i,'ShortCmpPerc'] <= 80):
      return 0
    elif (df_points.loc[i,'ShortCmpPerc'] > 80) & (df_points.loc[i,'ShortCmpPerc'] <= 90):
      return 0.025
    else:
      return 0.05
  elif (df_points.loc[i,'ShortAtt'] > 25) & (df_points.loc[i,'ShortAtt'] <= 50):
    if df_points.loc[i,'ShortCmpPerc'] <= 50:
      return -0.1
    elif (df_points.loc[i,'ShortCmpPerc'] > 50) & (df_points.loc[i,'ShortCmpPerc'] <= 70):
      return -0.05
    elif (df_points.loc[i,'ShortCmpPerc'] > 70) & (df_points.loc[i,'ShortCmpPerc'] <= 80):
      return 0
    elif (df_points.loc[i,'ShortCmpPerc'] > 80) & (df_points.loc[i,'ShortCmpPerc'] <= 90):
      return 0.05
    else:
      return 0.1
  else:
    if df_points.loc[i,'ShortCmpPerc'] <= 50:
      return -0.2
    elif (df_points.loc[i,'ShortCmpPerc'] > 50) & (df_points.loc[i,'ShortCmpPerc'] <= 70):
      return -0.1
    elif (df_points.loc[i,'ShortCmpPerc'] > 70) & (df_points.loc[i,'ShortCmpPerc'] <= 80):
      return 0
    elif (df_points.loc[i,'ShortCmpPerc'] > 80) & (df_points.loc[i,'ShortCmpPerc'] <= 90):
      return 0.1
    else:
      return 0.2

# PASES A MEDIA DISTANCIA
def mid_pass(df_points, i):
  if df_points.loc[i,'MedAtt'] <= 10:
    return 0
  elif (df_points.loc[i,'MedAtt'] > 10) & (df_points.loc[i,'MedAtt'] <= 25):
    if df_points.loc[i,'MedCmpPerc'] <= 50:
      return -0.05
    elif (df_points.loc[i,'MedCmpPerc'] > 50) & (df_points.loc[i,'MedCmpPerc'] <= 65):
      return -0.025
    elif (df_points.loc[i,'MedCmpPerc'] > 65) & (df_points.loc[i,'MedCmpPerc'] <= 75):
      return 0
    elif (df_points.loc[i,'MedCmpPerc'] > 75) & (df_points.loc[i,'MedCmpPerc'] <= 90):
      return 0.025
    else:
      return 0.05
  else:
    if df_points.loc[i,'MedCmpPerc'] <= 50:
      return -0.1
    elif (df_points.loc[i,'MedCmpPerc'] > 50) & (df_points.loc[i,'MedCmpPerc'] <= 65):
      return -0.05
    elif (df_points.loc[i,'MedCmpPerc'] > 65) & (df_points.loc[i,'MedCmpPerc'] <= 75):
      return 0
    elif (df_points.loc[i,'MedCmpPerc'] > 75) & (df_points.loc[i,'MedCmpPerc'] <= 90):
      return 0.05
    else:
      return 0.1

# PASES LARGOS
def long_pass(df_points, i):
  if df_points.loc[i,'LongAtt'] <= 10:
    return 0
  else:
    if df_points.loc[i,'LongCmpPerc'] <= 50:
      return -0.1
    elif (df_points.loc[i,'LongCmpPerc'] > 65) & (df_points.loc[i,'LongCmpPerc'] <= 75):
      return 0
    elif (df_points.loc[i,'LongCmpPerc'] > 75) & (df_points.loc[i,'LongCmpPerc'] <= 90):
      return 0.05
    else:
      return 0.1

## CALCULATE POINTS
def calculate_points(df_points, i, df_att_values):
  punt_res = result(df_points, i)
  punt_tot = tot_pass(df_points, i)
  punt_short = short_pass(df_points, i)
  punt_mid = mid_pass(df_points, i)
  punt_long = long_pass(df_points, i)
  df_new.at[i, 'Puntuacion'] += np.sum(
      np.multiply(list(df_points.iloc[i][df_att_values.index]), 
                  list(df_att_values.Values)))
  df_new.at[i,'Puntuacion'] += punt_res + punt_tot + punt_short + punt_mid + punt_long
  return df_new.at[i,'Puntuacion']

In [ ]:
# Crear variable puntuación
def create_var_points(df_new):
  df_points = df_new[['Player', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Squad', 
                     'Opponent', 'Start', 'Pos', 'Min', 'Result', 'nPG', 'PK',
                     'Ast', 'PKMissed', 'Sh', 'SoT', 'TotAtt', 'TotCmpPerc',
                     'PrgDist', 'ShortAtt', 'ShortCmpPerc', 'MedAtt',
                     'MedCmpPerc', 'LongAtt', 'LongCmpPerc', 'KP', 'SCA', 'GCA',
                     'TklW', 'PressSucc', 'Blocks', 'Int', 'Clr', 'Err', 
                     'SuccDrib', 'PrgDistCarries', 'ProgPassRec', 'CrdY', 'CrdR', 
                     'FlsComm', 'FlsDrawn', 'Offsides', 'PKwon', 'PKcon', 'OG', 
                     'Recov', 'AerialDuelsWon', 'AerialDuelsLost']]
  df_new.loc[:, 'Puntuacion'] = 5.0
  df_att_values = pd.DataFrame(data = df_points.columns).rename(columns={0: 'Attributes'})
  df_att_values['Values'] = 0
  df_att_values.loc[df_att_values['Attributes'] == 'Result','Values'] = 0.3
  df_att_values.loc[df_att_values['Attributes'] == 'nPG','Values'] = 0.75
  df_att_values.loc[df_att_values['Attributes'] == 'PK','Values'] = 0.3
  df_att_values.loc[df_att_values['Attributes'] == 'Ast','Values'] = 0.6
  df_att_values.loc[df_att_values['Attributes'] == 'PKMissed','Values'] = -0.6
  df_att_values.loc[df_att_values['Attributes'] == 'Sh','Values'] = 0.1
  df_att_values.loc[df_att_values['Attributes'] == 'SoT','Values'] = 0.05
  df_att_values.loc[df_att_values['Attributes'] == 'PrgDist','Values'] = 0.001
  df_att_values.loc[df_att_values['Attributes'] == 'KP','Values'] = 0.05
  df_att_values.loc[df_att_values['Attributes'] == 'SCA','Values'] = 0.075
  df_att_values.loc[df_att_values['Attributes'] == 'GCA','Values'] = 0.175
  df_att_values.loc[df_att_values['Attributes'] == 'TklW','Values'] = 0.05
  df_att_values.loc[df_att_values['Attributes'] == 'PressSucc','Values'] = 0.025
  df_att_values.loc[df_att_values['Attributes'] == 'Blocks','Values'] = 0.05
  df_att_values.loc[df_att_values['Attributes'] == 'Int','Values'] = 0.05
  df_att_values.loc[df_att_values['Attributes'] == 'Clr','Values'] = 0.05
  df_att_values.loc[df_att_values['Attributes'] == 'Err','Values'] = -1
  df_att_values.loc[df_att_values['Attributes'] == 'SuccDrib','Values'] = 0.1
  df_att_values.loc[df_att_values['Attributes'] == 'PrgDistCarries','Values'] = 0.001
  df_att_values.loc[df_att_values['Attributes'] == 'ProgPassRec','Values'] = 0.01
  df_att_values.loc[df_att_values['Attributes'] == 'CrdY','Values'] = -0.3
  df_att_values.loc[df_att_values['Attributes'] == 'CrdR','Values'] = -0.5
  df_att_values.loc[df_att_values['Attributes'] == 'FlsComm','Values'] = -0.1
  df_att_values.loc[df_att_values['Attributes'] == 'FlsDrawn','Values'] = 0.1
  df_att_values.loc[df_att_values['Attributes'] == 'Offsides','Values'] = -0.05
  df_att_values.loc[df_att_values['Attributes'] == 'PKwon','Values'] = 0.5
  df_att_values.loc[df_att_values['Attributes'] == 'PKcon','Values'] = -0.5
  df_att_values.loc[df_att_values['Attributes'] == 'OG','Values'] = -1
  df_att_values.loc[df_att_values['Attributes'] == 'Recov','Values'] = 0.05
  df_att_values.loc[df_att_values['Attributes'] == 'AerialDuelsWon','Values'] = 0.05
  df_att_values.loc[df_att_values['Attributes'] == 'AerialDuelsLost','Values'] = -0.03
  df_att_values = df_att_values[~df_att_values['Attributes'].isin(
      ['Player', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Squad', 'Opponent', 
       'Start', 'Pos', 'Min', 'TotAtt', 'TotCmpPerc', 'ShortAtt', 'ShortCmpPerc', 
       'MedAtt', 'MedCmpPerc', 'LongAtt', 'LongCmpPerc', 
       'Result', 'Puntuacion'])].set_index('Attributes')
  df_points = df_points.reset_index(drop=True)

  for i in range(len(df_points)):
    df_new.at[i, 'Puntuacion'] = calculate_points(df_points, i, df_att_values)
  return df_new['Puntuacion']

# Obtención del dato

In [ ]:
# df = pd.DataFrame()
# for player in Global.DICT_PLAYERS:
#   print(player)
#   for season in Global.SEASONS:
#     df_new = scrape_all(player, season, Global.DICT_PLAYERS[player], Global.TYPES)
#     df = pd.concat([df, df_new], ignore_index=True)
# df = df.fillna(0)
# df.to_csv('df.csv')
df = pd.read_csv('/content/drive/My Drive/MasterIADeporte/TFM/DatosFinales/df.csv')
df

# Procesamiento del dato

## Primer análisis exploratorio de datos

In [ ]:
profile = pandas_profiling.ProfileReport(df, minimal=True)
profile

## Ingeniería de variables

Definir nuevas variables, seleccionar entre las existentes y borrar campos con muchos valores vacíos.

In [ ]:
df_new = df.dropna(subset=['xG']).reset_index(drop=True)
df_new['nPG'] = df_new['Gls'] - df_new['PK']
df_new['PKMissed'] = df_new['PKatt'] - df_new['PK']
df_new['xPK'] = df_new['xG'] - df_new['npxG']
df_new['Puntuacion'] = create_var_points(df_new)
df_result = pd.get_dummies(df_new.Result, prefix='Result')
df_result[['Player', 'Date']] = df_new[['Player', 'Date']]
df_final = df_new.merge(df_result, how='inner', on=['Player', 'Date']).drop(
    columns = ['Date', 'Day', 'Comp', 'Round', 'Venue', 'Squad', 'Opponent', 
               'Pos', 'TotCmpPerc', 'ShortCmpPerc', 'MedCmpPerc', 'LongCmpPerc', 
               'TklDrib', 'Squad_Goals', 'Opponent_Goals',
               'Gls', 'PKatt', 'xG', 'Att', 'InCK', 'OutCK', 'StrCK', 'Other', 
               'CmpPass', 'SCA', 'GCA', 'TklDribPerc', 'PressPerc', 
               'PassBlock', 'Tkl+Int', 'Touches', 'SuccDribPerc', 'RecPassPerc', 
               'AerialDuelsWinPerc', 'Puntuacion', 'Result', 'Result_D']).fillna(0)
df_final

## Agregar datos por jugador
Generar variables de porcentaje. Borrar variables de intentos relacionadas con dichos porcentajes (estarían correladas).

In [ ]:
df_agg = df_final.groupby(['Player']).mean().rename(
                   columns={'Result_L': 'LosePerc', 'Result_W': 'WinPerc',
                            'Start': 'StartPerc'})
df_agg['TotCmpPerc'] = df_agg['TotCmp']/df_agg['TotAtt']
df_agg['ShortCmpPerc'] = df_agg['ShortCmp']/df_agg['ShortAtt']
df_agg['MedCmpPerc'] = df_agg['MedCmp']/df_agg['MedAtt']
df_agg['LongCmpPerc'] = df_agg['LongCmp']/df_agg['LongAtt']
df_agg['TklWPerc'] = df_agg['TklW']/df_agg['Tkl']
df_agg['TklDribPerc'] = df_agg['TklDribPast']/df_agg['TklDribAtt']
df_agg['PressPerc'] = df_agg['PressSucc']/df_agg['PressAtt']
df_agg['SuccDribPerc'] = df_agg['SuccDrib']/df_agg['AttDrib']
df_agg['RecPassPerc'] = df_agg['RecPass']/df_agg['TargPass']
df_agg['AerialDuelsPerc'] = df_agg['AerialDuelsWon']/df_agg['AerialDuelsLost']
df_agg = df_agg.drop(columns = ['TotAtt', 'ShortAtt', 'MedAtt', 'LongAtt', 
                                'Tkl', 'TklDribAtt', 'PressAtt', 'AttDrib', 
                                'TargPass', 'AerialDuelsLost'])
df_agg

## Segundo análisis exploratorio de datos

In [ ]:
profile = pandas_profiling.ProfileReport(df_agg, minimal=True)
profile

## Normalizar datos

In [ ]:
x_scaled = preprocessing.MinMaxScaler().fit_transform(df_agg.values)
df_norm = pd.DataFrame(x_scaled, index=df_agg.index, columns=df_agg.columns)
df_norm

## Tercer análisis exploratorio de datos

In [ ]:
profile = pandas_profiling.ProfileReport(df_norm, minimal=True)
profile

# Modelización del dato

## Primer algoritmo

In [ ]:
plt.figure(figsize=(30, 8))
plt.title('Dendograma')
plt.xlabel('Jugadores')
plt.ylabel('Distancias Euclidianas')
dendrogram = sch.dendrogram(sch.linkage(df_norm.values, method = 'ward'),
                            labels=df_norm.index, leaf_font_size=12.,
                            color_threshold=4)
plt.show()

## Segundo algoritmo

In [ ]:
plt.figure(figsize=(50, 10))
plt.title('Dendograma')
plt.xlabel('Jugadores')
plt.ylabel('Distancias Euclidianas')
dendrogram = sch.dendrogram(sch.linkage(df_norm.values, method = 'complete'),
                            labels=df_norm.index, leaf_font_size=12.,
                            color_threshold=4)
plt.show()

## Tercer algoritmo

In [ ]:
## K-MEANS
clustering = KMeans(n_clusters=5, random_state=1234).fit(df_norm)
# 2-COMPONENT PCA
pca = PCA(n_components=2, random_state=1234)
pca.fit(df_norm.values)
pca_vals = pca.transform(df_norm.values)
df_pca = pd.DataFrame(pca_vals, columns=['PC1','PC2'])
# PLOT K-MEANS
plt.figure(figsize=(20, 10))
plt.scatter(df_pca['PC1'], df_pca['PC2'], c=clustering.labels_, cmap='rainbow', 
            alpha=0.7, edgecolors='b')
for i, txt in enumerate(df_norm.index):
    plt.annotate(txt, (df_pca['PC1'][i], df_pca['PC2'][i]))
plt.show()
print(pca.explained_variance_ratio_)
df_comp = pd.DataFrame(pca.components_, columns=df_norm.columns, index=['PC1', 'PC2'])
df_comp

## Cuarto algoritmo

In [ ]:
## GMM
gmm = GaussianMixture(n_components=5, random_state=1234).fit(df_norm.values)
labels = gmm.predict(df_norm.values)
## 2-COMPONENT PCA
pca = PCA(n_components=2, random_state=1234)
pca.fit(df_norm.values)
X_pca_array = pca.transform(df_norm.values)
X_pca = pd.DataFrame(X_pca_array, columns=['PC1','PC2'])
## PLOT GMM
plt.figure(figsize=(20, 10))
plt.scatter(X_pca['PC1'], X_pca['PC2'], c=labels, cmap='rainbow', 
            alpha=0.7, edgecolors='b')
for i, txt in enumerate(df_norm.index):
    plt.annotate(txt, (X_pca['PC1'][i], X_pca['PC2'][i]))
plt.show()
print(pca.explained_variance_ratio_)
df_comp = pd.DataFrame(pca.components_, columns=df_norm.columns, index=['PC1', 'PC2'])
df_comp